# Avito LGBM (wo images)

## Install LGBM

In [ ]:
! df

In [ ]:
%bash
mkdir /dev/sdb/lgbm
pip install lightgbm -t /dev/sdb/lgbm

In [ ]:
!ls /dev/sdb/lgbm/lightgbm

In [ ]:
import sys
sys.path.append('/dev/sdb/lgbm/')

## Load data

In [ ]:
#import google.datalab.bigquery as bq
import pandas as pd

In [ ]:
! gsutil cp gs://kaggle893/avito/sample_submission.csv ./

gcsに保存したcsvをローカルにコピーして読み混む

In [ ]:
! gsutil cp gs://kaggle893/avito/feature/_8th_feature_macros*.gz ./
! gsutil cp gs://kaggle893/avito/feature/train_test_uq_rate.csv ./
! gsutil cp gs://kaggle893/avito/bi_lgbm_gru_4f_seed7/* ./
! gsutil cp gs://kaggle893/avito/feature/numel_text.csv ./
! gsutil cp gs://kaggle893/avito/feature/text_ftrs.csv ./
! gsutil cp gs://kaggle893/avito/feature/ridge_oof.csv ./
! gsutil cp gs://kaggle893/avito/feature/stddev_price_image_top_1_000000000000.gz ./
! gsutil cp gs://kaggle893/avito/feature/_stddev_week_dow_day_by_user_id.csv ./

In [ ]:
import os

path_list = os.listdir("./")
train_csvs = [path for path in path_list if "_8th_feature_macros" in path]

train_test_df = pd.DataFrame()
for csv_part in train_csvs:
  tmp_df = pd.read_csv(csv_part, dtype={"param_1" : object, "param_2" : object, "param_3" : object})
  train_test_df = train_test_df.append(tmp_df)

In [ ]:
train_test_df.head()

## Merge src data and oof data

In [ ]:
import numpy as np
oof_X_train = np.load("bi_lstm_gru_ftext_neo_wiki_cc_oof_X_train_4f_seed7.npy").reshape(-1,)
oof_X_test = pd.read_csv("bi_lstm_gru_ftext_neo_wiki_cc_4f_seed7.csv")
oof_item_id_train = pd.read_csv("oof_item_id_train.csv")

concatenate item_id and oofs

In [ ]:
oof_item_id_train["deal_probability"] = oof_X_train
oof_X_train_test = oof_item_id_train.append(oof_X_test).reset_index()
oof_X_train_test = oof_X_train_test.rename(columns={'deal_probability': 'oof'})

In [ ]:
train_test_df = pd.merge(train_test_df, oof_X_train_test[["item_id", "oof"]], how='left')
train_test_df.head()

## Merge src data and unique rate from bi_lstm_gru model

In [ ]:
train_test_uq_rate = pd.read_csv("train_test_uq_rate.csv")

In [ ]:
train_test_df = pd.merge(train_test_df, train_test_uq_rate[["item_id", "unique_rate"]], how='left')
train_test_df.head()

## Merge src data and numelical value from texts

In [ ]:
numel_text = pd.read_csv("numel_text.csv")

In [ ]:
train_test_df = pd.merge(train_test_df, numel_text[list(numel_text.columns)], how='left')
train_test_df.head()

## Merge src data and text features

In [ ]:
text_ftrs = pd.read_csv("text_ftrs.csv")

In [ ]:
train_test_df = pd.merge(train_test_df, text_ftrs[list(text_ftrs.columns)], how='left')
train_test_df.head()

## Merge src data and stddev_price_image_top_1 features

In [ ]:
stddev_pr_imt1 = pd.read_csv("stddev_price_image_top_1_000000000000.gz")

In [ ]:
train_test_df = pd.merge(train_test_df, stddev_pr_imt1[list(stddev_pr_imt1.columns)], how='left')
train_test_df.head()

## Merge src data and stddev_time_by_user_id features

In [ ]:
stddev_time = pd.read_csv("_stddev_week_dow_day_by_user_id.csv")

In [ ]:
train_test_df = pd.merge(train_test_df, stddev_time[list(stddev_time.columns)], how='left')
train_test_df.head()

## Merge src data and ridge oof

In [ ]:
ridge_oof = pd.read_csv("ridge_oof.csv")

In [ ]:
train_test_df = pd.merge(train_test_df, ridge_oof[["item_id","ridge_oof"]], how='left')
train_test_df.head()

## Log Transforms

Functions

In [ ]:
def log2_transform(df, columns):
  for fea in columns:  
    df[fea]= np.log2(1 + df[fea].values).astype(int)
  return df

In [ ]:
def log10_transform(df, columns):
  for fea in columns:  
    df[fea]= np.log10(1 + df[fea].values).astype(int)
  return df

Apply log transformation

## Label encoding

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
#train_test_df[["param_1", "param_2", "param_3"]] = train_test_df[["param_1", "param_2", "param_3"]].to_string() # intを無理やり変換すると結果が著しく悪くなるぞ
train_test_df[["param_1", "param_2", "param_3"]] = train_test_df[["param_1", "param_2", "param_3"]].fillna("no_description")

In [ ]:
train_test_df[["user_id", "region", "city", "parent_category_name", "category_name", "param_1", "param_2", "param_3", "user_type"]] \
= train_test_df[["user_id", "region", "city", "parent_category_name", 
                 "category_name", "param_1", "param_2", "param_3", "user_type"]].apply(le.fit_transform)

In [ ]:
train_test_df.head()

### additional_1

In [ ]:
additional_cols = ['category_name_price_image_top_1_minmax_diff',
                   'city_category_name_image_top_1_minmax_diff',
                   'city_category_name_price_minmax_diff',
                   'city_param_1_category_name_minmax_diff',
                   'city_param_1_image_top_1_minmax_diff',
                   'city_param_1_param_2_minmax_diff',
                   'city_param_1_param_3_minmax_diff',
                   'city_param_1_price_minmax_diff',
                   'city_param_2_category_name_minmax_diff',
                   'city_param_2_image_top_1_minmax_diff',
                   'city_param_2_param_3_minmax_diff',
                   'city_param_2_price_minmax_diff',
                   'city_param_3_category_name_minmax_diff',
                   'city_param_3_image_top_1_minmax_diff',
                   'city_price_image_top_1_minmax_diff',
                   'param_1_category_name_image_top_1_minmax_diff',
                   'param_1_category_name_price_minmax_diff',
                   'param_1_param_2_category_name_minmax_diff',
                   'param_1_param_2_image_top_1_minmax_diff',
                   'param_1_param_2_param_3_minmax_diff',
                   'param_1_param_2_price_minmax_diff',
                   'param_1_param_3_category_name_minmax_diff',
                   'param_1_param_3_image_top_1_minmax_diff',
                   'param_1_param_3_price_minmax_diff',
                   'param_1_price_image_top_1_diff',
                   'param_2_category_name_image_top_1_minmax_diff',
                   'param_2_category_name_price_diff',
                   'param_2_param_3_image_top_1_minmax_diff',
                   'param_2_param_3_price_minmax_diff',
                   'param_2_price_image_top_1_minmax_diff',
                   'param_3_category_name_image_top_1_minmax_diff',
                   'param_3_category_name_price_minmax_diff',
                   'param_3_price_image_top_1_minmax_diff',
                   'user_id_category_name_image_top_1_minmax_diff',
                   'user_id_category_name_price_minmax_diff',
                   'user_id_city_category_name_minmax_diff',
                   'user_id_city_image_top_1_minmax_diff',
                   'user_id_city_param_1_minmax_diff',
                   'user_id_city_param_2_minmax_diff',
                   'user_id_city_param_3_minmax_diff',
                   'user_id_city_price_minmax_diff',
                   'user_id_param_1_category_name_minmax_diff',
                   'user_id_param_1_image_top_1_minmax_diff',
                   'user_id_param_1_param_2_minmax_diff',
                   'user_id_param_1_param_3_minmax_diff',
                   'user_id_param_1_price_minmax_diff',
                   'user_id_param_2_category_name_minmax_diff',
                   'user_id_param_2_image_top_1_minmax_diff',
                   'user_id_param_2_param_3_minmax_diff',
                   'user_id_param_2_price_minmax_diff',
                   'user_id_param_3_category_name_minmax_diff',
                   'user_id_param_3_image_top_1_minmax_diff',
                   'user_id_param_3_price_minmax_diff',
                   'user_id_price_image_top_1_minmax_diff',
                   'category_name_price_image_top_1_uq',
                   'city_category_name_image_top_1_uq',
                   'city_category_name_price_uq',
                   'city_param_1_category_name_uq',
                   'city_param_1_image_top_1_uq',
                   'city_param_1_param_2_uq',
                   'city_param_1_param_3_uq',
                   'city_param_1_price_uq',
                   'city_param_2_category_name_uq',
                   'city_param_2_image_top_1_uq',
                   'city_param_2_param_3_uq',
                   'city_param_2_price_uq',
                   'city_param_3_category_name_uq',
                   'city_param_3_image_top_1_uq',
                   'city_price_image_top_1_uq',
                   'param_1_category_name_image_top_1_uq',
                   'param_1_category_name_price_uq',
                   'param_1_param_2_category_name_uq',
                   'param_1_param_2_image_top_1_uq',
                   'param_1_param_2_param_3_uq',
                   'param_1_param_2_price_uq',
                   'param_1_param_3_category_name_uq',
                   'param_1_param_3_image_top_1_uq',
                   'param_1_param_3_price_uq',
                   'param_1_price_image_top_1_uq',
                   'param_2_category_name_image_top_1_uq',
                   'param_2_category_name_price_uq',
                   'param_2_param_3_image_top_1_uq',
                   'param_2_param_3_price_uq',
                   'param_2_price_image_top_1_uq',
                   'param_3_category_name_image_top_1_uq',
                   'param_3_category_name_price_uq',
                   'param_3_price_image_top_1_uq',
                   'user_id_category_name_image_top_1_uq',
                   'user_id_category_name_price_uq',
                   'user_id_city_category_name_uq',
                   'user_id_city_image_top_1_uq',
                   'user_id_city_param_1_uq',
                   'user_id_city_param_2_uq',
                   'user_id_city_param_3_uq',
                   'user_id_city_price_uq',
                   'user_id_param_1_category_name_uq',
                   'user_id_param_1_image_top_1_uq',
                   'user_id_param_1_param_2_uq',
                   'user_id_param_1_param_3_uq',
                   'user_id_param_1_price_uq',
                   'user_id_param_2_category_name_uq',
                   'user_id_param_2_image_top_1_uq',
                   'user_id_param_2_param_3_uq',
                   'user_id_param_2_price_uq',
                   'user_id_param_3_category_name_uq',
                   'user_id_param_3_image_top_1_uq',
                   'user_id_param_3_price_uq',
                   'user_id_price_image_top_1_uq']

In [ ]:
train_test_df[additional_cols] = train_test_df[additional_cols].fillna(0)

In [ ]:
for idx, col in enumerate(stddev_pr_imt1.columns[1:]):
  col_mean = train_test_df[col].mean() 
  train_test_df[col] = train_test_df[col].fillna(col_mean)
  sys.stdout.write("\r {:d} of {:d}".format(idx+1, len(stddev_pr_imt1.columns[1:])))

### additional_2
0.0が存在しているせいで-1を埋めている（label encodeするしかないのか？）

-1は経験的にスコア悪くすることがわかっているので，0に振るか one_hot_encodingにかえた方がよいかも<br>
**Kernelsでは`-999`を充てている**

In [ ]:
train_test_df["image_top_1"] = train_test_df["image_top_1"].fillna(-1)
#train_test_df[["image_top_1"]] = train_test_df[["image_top_1"]].apply(le.fit_transform)

In [ ]:
train_test_df.head()

## train-test split

In [ ]:
train_df = train_test_df[~train_test_df["deal_probability"].isnull()].reset_index(drop=True)
test_df = train_test_df[train_test_df["deal_probability"].isnull()].reset_index(drop=True)

## Eliminate specific samples for train_data

In [ ]:
!gsutil cp gs://kaggle893/avito/feature/elim_item_id.csv ./

In [ ]:
elim_item_id = pd.read_csv("elim_item_id.csv")
elim_item_id.head()

In [ ]:
import sys
elim_idx_list = []
for idx, item in enumerate(elim_item_id["item_id"]):
  elim_idx = train_df[train_df["item_id"] == item].index
  if len(elim_idx) != 0:
    elim_idx_list.append(list(elim_idx)[0])
  sys.stdout.write("\r {:d}".format(idx+1))
  sys.stdout.flush()


In [ ]:
train_df = train_df.drop(train_df.index[elim_idx_list]).reset_index(drop=True)
train_df.head()

## LGBM settings

In [ ]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
import gc
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import KFold

In [ ]:
def lgb_modelfit(params, dtrain, dvalid, predictors, ytrain, yvalid, objective='regression', metrics='rmse',
                 feval=None, early_stopping_rounds=20, num_boost_round=3000, verbose_eval=10, categorical_features=None):
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': objective,
        'metric': metrics,
        'learning_rate': 0.2,
        #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
        'num_leaves': 31,  # we should let it be smaller than 2^(max_depth)
        'max_depth': -1,  # -1 means no limit
        'min_child_samples': 20,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 255,  # Number of bucketed bin for feature values
        'subsample': 0.6,  # Subsample ratio of the training instance.
        'subsample_freq': 0,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.3,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 5,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 10 ** (-1),  # L1 regularization term on weights; 0
        'reg_lambda': 10 ** (-1),  # L2 regularization term on weights: 0
        'nthread': 4,
        'verbose': 0,
        'metric':metrics
    }

    lgb_params.update(params)

    print("preparing validation datasets")

    xgtrain = lgb.Dataset(dtrain[predictors].values, label=ytrain,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )
    xgvalid = lgb.Dataset(dvalid[predictors].values, label=yvalid,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )

    evals_results = {}

    bst1 = lgb.train(lgb_params, 
                     xgtrain, 
                     valid_sets=[xgtrain, xgvalid], 
                     valid_names=['train','valid'], 
                     evals_result=evals_results, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10, 
                     feval=feval)

    print("\nModel Report")
    print("bst1.best_iteration: ", bst1.best_iteration)
    
    # generate oof_X_train
    oof_X_train = bst1.predict(dvalid[predictors], num_iteration=bst1.best_iteration)

    return (bst1, bst1.best_iteration, oof_X_train)

def DO(userows, train_df, test_df, sub_df, predictors, categoricals, debug=0, seed=7, fold_num=4, file_path="./lgbm_outs/"):
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    print('loading train data...')
    if debug:
        print('*** debug parameter set: this is a test run for debugging purposes ***')
        train_df = train_df[:userows]
    len_train = len(train_df)
     
    print('predictors', predictors)
    print('categoricals', categoricals)
    
    
    # define K-fold cross validation
    kfold = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # Initialize oof_xtrain
    oof_X_train = []
    # initialize output
    sum_output =[]
    
    
    print("Training {:d}fold seed{:d}".format(fold_num, seed))
    start_time = time.time()

    y_train = train_df["deal_probability"].values
    for fold_idx, (train_idx, eval_idx) in enumerate(kfold.split(train_df)):
        # Get train-valid splited dataset
        val_df = train_df.iloc[eval_idx]
        train_df_kf =train_df.iloc[train_idx]
        y_train_kf = y_train[train_idx]
        y_eval_kf = y_train[eval_idx]

        print("train size: ", len(train_df_kf))
        print("valid size: ", len(val_df))
        print("test size : ", len(test_df))

        gc.collect()

        params = {
            'learning_rate': 0.1,  # 0.016 (本番用)
            #'is_unbalance': 'true', # replaced with scale_pos_weight argument
            'num_leaves': 15, #270 , 15(本番用), 7 # 2^max_depth - 1
            #'max_depth': 3,  # -1 means no limit
            'feature_fraction': 0.5,
            'bagging_fraction': 0.75,
            'bagging_freq': 2,
            'min_child_samples': 64,  # Minimum number of data need in a child(min_data_in_leaf)
            'max_bin': 63,  # Number of bucketed bin for feature values
            'subsample': 0.7,  # Subsample ratio of the training instance.
            'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
            'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
            'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
            'scale_pos_weight': 200, # because training data is extremely unbalanced 
            'lambda_l1': 10 ** (-1),  # L1 regularization term on weights; 0  lambda_l1, reg_alpha
            'lambda_l2': 10 ** (-1)  # L2 regularization term on weights: 0 #-6  reg_lambda
        }
        (bst, best_iteration, oof_X_train_kf) = lgb_modelfit(params, 
                                                             train_df_kf, 
                                                             val_df, 
                                                             predictors, 
                                                             y_train_kf,
                                                             y_eval_kf,
                                                             objective='regression', 
                                                             metrics=['rmse'],
                                                             early_stopping_rounds=30, 
                                                             verbose_eval=True, 
                                                             num_boost_round=3000, 
                                                             categorical_features=categoricals)
        # concat oof_train
        if fold_idx == 0:
            oof_X_train = oof_X_train_kf
            oof_y_train = y_eval_kf
        else:
            oof_X_train = np.concatenate([oof_X_train, oof_X_train_kf], axis=0)
            oof_y_train = np.concatenate([y_eval_kf, y_eval_kf], axis=0)
        #print('Save delta of oof_X_train_kf and label data')
            
        print('[{}]: model training time'.format(time.time() - start_time))
        print('Plot feature importances...')
        f, ax = plt.subplots(figsize=[14,40])
        lgb.plot_importance(bst, max_num_features=200, ax=ax)
        plt.title(file_path + "LGBM Feature Importance {:d}f_seed{:d}".format(fold_idx, seed))
        plt.savefig(file_path + 'ftr_imp_{:d}f_seed{:d}.png'.format(fold_idx, seed))
        plt.show()

        print("Predicting...")
        if fold_idx == 0:
            sum_output = bst.predict(test_df[predictors],num_iteration=best_iteration)
        else:
            sum_output += bst.predict(test_df[predictors],num_iteration=best_iteration)
    
    
    print('Save out-of-fold X, y train array')
    np.save(file_path + 'LGBM_8th_{:d}f_seed{:d}.npy'.format(fold_num, seed), oof_X_train)
    np.save(file_path + 'LGBM_8th_{:d}f_seed{:d}.npy'.format(fold_num, seed), oof_y_train)
    # fold eval y_labelは他の弱学習器と同じなので保存しない
    
    del train_df, train_df_kf
    del val_df
    gc.collect()
    
    if not debug:
        print("writing...")
        outputs = sum_output / fold_num
        test_df["deal_probability"] = outputs
        sub_df = pd.merge(sub_df[["item_id"]], test_df[["item_id", "deal_probability"]], how='left')
        # [0, 1]からはみ出した値を丸める
        sub_df["deal_probability"] = sub_df["deal_probability"].apply(lambda x: 0. if x < 0 else x)
        sub_df["deal_probability"] = sub_df["deal_probability"].apply(lambda x: 1. if x > 1 else x)
        sub_df[["item_id", "deal_probability"]].to_csv(file_path + 'LGBM_8th_{:d}f_seed{:d}.csv'.format(fold_num, seed),index=False)
    print("done...")

In [ ]:
list(train_df.columns)

In [ ]:
# 注：day_diffは罠なので，含めないようにする．その他の変数は可視化して罠かどうか確認すること
# Upadate: 2018-06-19 price -> price_fn (fillna:AVG(price)), image_top_1 -> image_top_1 (fillna:-999)

predictors = ['user_id',
             'region',
             'city',
             'parent_category_name',
             'category_name',
             'param_1',
             'param_2',
             'param_3',
             'user_type',
             'price',
             'image_top_1',
             'oof',
             #'ridge_oof',
             'unique_rate',
             'week',
             'dayofweek',
             'day',
             'population',
             #'unemployment_rate', 
             #'GDP_PC_PPP', 
             #'HDI',
             'all_categ_minmax_diff',
             'category_name_unique',
             'all_category_count',
             'city_unique',
             'region_city_count',
             'region_city_all_category_count',
             'all_categ_minmax_diff_image_top_1_count',
             'category_name_all_categ_minmax_diff_count',
             'category_name_count',
             'category_name_image_top_1_count',
             'category_name_param_2_count',
             'category_name_param_3_count',
             'category_name_price_count',
             'category_name_user_type_count',
             'city_all_categ_minmax_diff_count_count',
             'city_category_name_count',
             'city_count',
             'city_param_1_count',
             'city_param_3_count',
             'city_parent_category_name_count',
             'city_price_count',
             'city_user_type_count',
             'image_top_1_count',
             'param_1_all_categ_minmax_diff_count',
             'param_1_count',
             'param_1_image_top_1_count',
             'param_1_param_2_count',
             'param_1_param_3_count',
             'param_1_price_count',
             'param_1_user_type_count',
             'param_2_count',
             'param_2_image_top_1_count',
             'param_2_param_3_count',
             'param_2_price_count',
             'param_2_user_type_count',
             'param_3_all_categ_minmax_diff_count',
             'param_3_count',
             'param_3_image_top_1_count',
             'param_3_price_count',
             'param_3_user_type_count',
             'parent_category_name_all_categ_minmax_diff_count',
             'parent_category_name_child_category_name_count',
             'parent_category_name_count',
             'parent_category_name_image_top_1_count',
             'parent_category_name_param_1_count',
             'parent_category_name_param_2_count',
             'parent_category_name_param_3_count',
             'parent_category_name_price_count',
             'parent_category_name_user_type_count',
             'parent_child_category_name_unique',
             'parent_child_param_1_2_3_count',
             'price_all_categ_minmax_diff_count',
             'price_count',
             'price_image_top_1_count',
             'region_all_categ_minmax_difff_count',
             'region_child_category_name_count',
             'region_city_unique',
             'region_count',
             'region_image_top_1_count',
             'region_param_1_count',
             'region_param_2_count',
             'region_param_3_count',
             'region_parent_category_name_count',
             'region_price_count',
             'region_user_type_count',
             'user_id_all_categ_minmax_diff_count',
             'user_id_child_category_name_count',
             'user_id_city_count',
             'user_id_count',
             'user_id_image_top_1_count',
             'user_id_param_1_count',
             'user_id_param_2_count',
             'user_id_param_3_count',
             'user_id_parent_category_name_count',
             'user_id_region_count',
             'user_id_user_type_count',
             'user_type_all_categ_minmax_diff_count',
             'user_type_count',
             'user_type_image_top_1_count',
             'user_type_price_count',
             'category_name_minmax_diff',
             'city_minmax_diff',
             'image_top_1_minmax_diff',
             'param_1_minmax_diff',
             'param_2_minmax_diff',
             'param_3_minmax_diff',
             'parent_category_name_minmax_diff',
             'price_minmax_diff',
             'region_minmax_diff',
             'user_id_minmax_diff',
             'user_type_minmax_diff',
             'category_name_price_image_top_1_count',
             'city_category_name_image_top_1_count',
             'city_category_name_price_count',
             'city_param_1_category_name_count',
             'city_param_1_image_top_1_count',
             'city_param_1_param_2_count',
             'city_param_1_param_3_count',
             'city_param_1_price_count',
             'city_param_2_category_name_count',
             'city_param_2_image_top_1_count',
             'city_param_2_param_3_count',
             'city_param_2_price_count',
             'city_param_3_category_name_count',
             'city_param_3_image_top_1_count',
             'city_param_3_price_count',
             'city_price_image_top_1_count',
             'param_1_category_name_image_top_1_count',
             'param_1_category_name_price_count',
             'param_1_param_2_category_name_count',
             'param_1_param_2_image_top_1_count',
             'param_1_param_2_param_3_count',
             'param_1_param_2_price_count',
             'param_1_param_3_category_name_count',
             'param_1_param_3_image_top_1_count',
             'param_1_param_3_price_count',
             'param_2_category_name_image_top_1_count',
             'param_2_category_name_price_count',
             'param_2_param_3_category_name_count',
             'param_2_param_3_image_top_1_count',
             'param_2_param_3_price_count',
             'param_2_price_image_top_1_count',
             'param_3_category_name_image_top_1_count',
             'param_3_category_name_price_count',
             'param_3_price_image_top_1_count',
             'user_id_category_name_image_top_1_count',
             'user_id_city_category_name_count',
             'user_id_city_image_top_1_count',
             'user_id_city_param_1_count',
             'user_id_city_param_2_count',
             'user_id_city_param_3_count',
             'user_id_city_price_count',
             'user_id_param_1_category_name_count',
             'user_id_param_1_image_top_1_count',
             'user_id_param_1_param_2_count',
             'user_id_param_1_param_3_count',
             'user_id_param_1_price_count',
             'user_id_param_2_category_name_count',
             'user_id_param_2_image_top_1_count',
             'user_id_param_2_param_3_count',
             'user_id_param_2_price_count',
             'user_id_param_3_category_name_count',
             'user_id_param_3_image_top_1_count',
             'user_id_param_3_price_count',
             'user_id_price_image_top_1_count',
             'category_name_price_image_top_1_uq',
             'city_category_name_image_top_1_uq',
             'city_category_name_price_uq',
             'city_param_1_category_name_uq',
             'city_param_1_image_top_1_uq',
             'city_param_1_param_2_uq',
             'city_param_1_param_3_uq',
             'city_param_1_price_uq',
             'city_param_2_category_name_uq',
             'city_param_2_image_top_1_uq',
             'city_param_2_param_3_uq',
             'city_param_2_price_uq',
             'city_param_3_category_name_uq',
             'city_param_3_image_top_1_uq',
             'city_price_image_top_1_uq',
             'param_1_category_name_image_top_1_uq',
             'param_1_category_name_price_uq',
             'param_1_param_2_category_name_uq',
             'param_1_param_2_image_top_1_uq',
             'param_1_param_2_param_3_uq',
             'param_1_param_2_price_uq',
             'param_1_param_3_category_name_uq',
             'param_1_param_3_image_top_1_uq',
             'param_1_param_3_price_uq',
             'param_1_price_image_top_1_uq',
             'param_2_category_name_image_top_1_uq',
             'param_2_category_name_price_uq',
             'param_2_param_3_image_top_1_uq',
             'param_2_param_3_price_uq',
             'param_2_price_image_top_1_uq',
             'param_3_category_name_image_top_1_uq',
             'param_3_category_name_price_uq',
             'param_3_price_image_top_1_uq',
             'user_id_category_name_image_top_1_uq',
             'user_id_category_name_price_uq',
             'user_id_city_category_name_uq',
             'user_id_city_image_top_1_uq',
             'user_id_city_param_1_uq',
             'user_id_city_param_2_uq',
             'user_id_city_param_3_uq',
             'user_id_city_price_uq',
             'user_id_param_1_category_name_uq',
             'user_id_param_1_image_top_1_uq',
             'user_id_param_1_param_2_uq',
             'user_id_param_1_param_3_uq',
             'user_id_param_1_price_uq',
             'user_id_param_2_category_name_uq',
             'user_id_param_2_image_top_1_uq',
             'user_id_param_2_param_3_uq',
             'user_id_param_2_price_uq',
             'user_id_param_3_category_name_uq',
             'user_id_param_3_image_top_1_uq',
             'user_id_param_3_price_uq',
             'user_id_price_image_top_1_uq']
# Add numelical features from texts and text features 
# numel_text_は特徴の変化が200列台後半に出るので，後半のみpredictorsに入れれば十分か検証
predictors = predictors + list(numel_text.columns)[231:] + list(text_ftrs.columns)[1:] + list(stddev_pr_imt1.columns)[1:] #+ list(stddev_time.columns)[1:]

            
categoricals = ['user_id', 'region', 'city', 'parent_category_name', 'category_name', 'param_1', 'param_2', 'param_3', 'user_type', 'image_top_1']
  
sub_df = pd.read_csv("./sample_submission.csv")
userows = 1000000
DO(userows, train_df, test_df, sub_df, predictors, categoricals, debug=0, seed=7, fold_num=4, file_path="./lgbm_8th_upper_debug_4f_seed7/")

## Check submission data

In [ ]:
lgbm_sub = pd.read_csv("./lgbm_8th_drop_upper4-29_debug_4f_seed7/LGBM_8th_4f_seed7.csv")
lgbm_sub 

## Copy the directory which contains output files to GCS

In [ ]:
! gsutil cp -r ./lgbm_8th_drop_upper4-29_debug_4f_seed7 gs://kaggle893/avito/